## Reinforcement Learning

Lernen eines Prozesses **ohne** vorgegebem Datenset

Während des Ablaufs des Prozesses werden die Daten und direkt verarbeitet

Der Agent (der Spieler) bekommt bei jedem Schritt des Prozesses ein Feedback in Form einer Belohnung, anhand dieser Belohnung passiert das Lernen

Begriffe
- Umgebung
    - Definiert den Prozess + Variablen innerhalb des Prozesses

- Agent
    - Der Spieler, welcher die Umgebung ausführt und die Belohnung erhält
 
Packages: gym(nasium), stable-baselines3

### Vordefinierte Umgebungen

Es gibt von OpenAI verschiedene vorgegebene Umgebungen

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [2]:
import gymnasium as gym

In [14]:
env = gym.make("CartPole-v1", render_mode="human")

### Definition des Agenten

Der Agent sammelt während des Prozesses Punkte, und kann dadurch bewertet werden

Inhalte einer Umgebung:
- \_\_init__: Definiert den Initialzustand der Umgebung
- step: Führt einen Schritt der Umgebung, hier wird auch die Belohnung ausgegeben
- reset: Setzt zu Umgebung zurück
- render: Zeichnet die Umgebung (wenn vorhanden)
- close: Schließt die Umgebung (z.B. GUI beenden)

Action Space: Aktionen die der Agent tätigen kann

Observation Space: Daten die das Spiel definieren

In [11]:
env.action_space.sample()

1

In [12]:
env.observation_space.sample()

array([-2.2306657e+00,  2.9734341e+38, -2.4736844e-01, -1.0336525e+38],
      dtype=float32)

Der ML-Algorithmus (später) wählt anhand der 4 Werte im Observation Space eine Action aus (0 oder 1)

In [15]:
durchgaenge = 10
for x in range(durchgaenge):
    state = env.reset()  # Am Anfang von jedem Durchgang die Umgebung zurücksetzen
    score = 0  # Zähler für Belohnungen
    done = False  # Zeigt ob der Agent noch im Spiel ist

    while not done:
        env.render()
        action = env.action_space.sample()  # Random Action aus dem Action Space entnehmen
        state, reward, done, term, info = env.step(action)
        # state: Der Status des Spiels (Variablen)
        # reward: Belohnung dieses steps
        # done, term: Beendet das Spiel
        # info: Extra Daten (falls vorhanden)
        score += reward

    print(f"Durchgang {x}, Score: {score}")
env.close()

Durchgang 0, Score: 14.0
Durchgang 1, Score: 24.0
Durchgang 2, Score: 30.0
Durchgang 3, Score: 12.0
Durchgang 4, Score: 35.0
Durchgang 5, Score: 24.0
Durchgang 6, Score: 76.0
Durchgang 7, Score: 17.0
Durchgang 8, Score: 12.0
Durchgang 9, Score: 13.0


### RL Model trainieren

Random Actions sind nur zum Testen praktisch

Wir wollen jetzt ein Machine Learning Model definieren, welches das Spiel erlernen kann

In [3]:
from stable_baselines3 import PPO

In [17]:
env = gym.make("CartPole-v1")
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [18]:
model.learn(total_timesteps=100_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.7     |
|    ep_rew_mean     | 21.7     |
| time/              |          |
|    fps             | 935      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 24.6        |
|    ep_rew_mean          | 24.6        |
| time/                   |             |
|    fps                  | 473         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008259477 |
|    clip_fraction        | 0.0869      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00219    |
|    learning_rate        | 0.

In [19]:
model.save("Models/CartPole")

C:\Users\lk3\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [21]:
env = gym.make("CartPole-v1")

In [22]:
model = PPO.load("Models/CartPole.zip", env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [23]:
durchgaenge = 5
for x in range(durchgaenge):
    state = env.reset()  # Am Anfang von jedem Durchgang die Umgebung zurücksetzen
    score = 0  # Zähler für Belohnungen
    done = False  # Zeigt ob der Agent noch im Spiel ist
    state = state[0].reshape(1, 4)

    while not (done or term):
        # env.render()
        action, _ = model.predict(state)  # Random Action aus dem Action Space entnehmen
        if action.ndim == 1:
            action = action[0]
        
        state, reward, done, term, info = env.step(action)
        # state: Der Status des Spiels (Variablen)
        # reward: Belohnung dieses steps
        # done, term: Beendet das Spiel
        # info: Extra Daten (falls vorhanden)
        score += reward

    print(f"Durchgang {x}, Score: {score}")
env.close()

Durchgang 0, Score: 0
Durchgang 1, Score: 0
Durchgang 2, Score: 0
Durchgang 3, Score: 0
Durchgang 4, Score: 0
